In [1]:
pima=pd.read_csv('diabetes.csv').drop(['Outcome'],axis=1)

In [2]:
pima.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [3]:
pima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
dtypes: float64(2), int64(6)
memory usage: 48.1 KB


In [4]:
pima.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [5]:
pima.shape

(768, 8)

In [6]:
# 75% data are used for training purposes and rest are for testing purpose.
nobs=192
x_train,x_test=pima[:-nobs],pima[-nobs:]
print(x_train.shape)
print(x_test.shape)

(576, 8)
(192, 8)


In [7]:
from statsmodels.tsa.stattools import grangercausalitytests 

In [8]:
maxlag=1
test='ssr_chi2test'

In [9]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):
    X_train = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in X_train.columns:
        for r in X_train.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y={r},X={c},P Values = {p_values}')
            min_p_value=np.min(p_values)
            X_train.loc[r, c] = min_p_value
    X_train.columns =[var + '_x' for var in variables]
    X_train.index =  [var + '_y' for var in variables]
    return X_train


In [10]:
grangers_causation_matrix(x_train,variables=x_train.columns)

,Pregnancies_x,Glucose_x,BloodPressure_x,SkinThickness_x,Insulin_x,BMI_x,DiabetesPedigreeFunction_x,Age_x
Pregnancies_y,1.0000,0.6853,0.2617,0.3454,0.6378,0.6896,0.0561,0.9336
Glucose_y,0.0661,1.0000,0.9815,0.9471,0.6232,0.3647,0.9944,0.4096
BloodPressure_y,0.0125,0.5102,1.0000,0.9172,0.4214,0.5103,0.0758,0.2313
SkinThickness_y,0.3323,0.1895,0.2407,1.0000,0.3499,0.9750,0.6065,0.7099
Insulin_y,0.0521,0.0116,0.9418,0.2841,1.0000,0.7297,0.2817,0.1546
BMI_y,0.6890,0.2317,0.7314,0.2409,0.1008,1.0000,0.4136,0.7436
DiabetesPedigreeFunction_y,0.2007,0.0458,0.5534,0.9095,0.1443,0.9693,1.0000,0.8094
Age_y,0.1042,0.8715,0.4906,0.0507,0.1865,0.2883,0.0699,1.0000
